<a href="https://colab.research.google.com/github/dorobat-diana/LicentaAi/blob/main/EfficienNet_FamousPlacesDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check for CUDA available.

In [ ]:
import torch
print("GPU Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")


GPU Available: True
GPU Name: Tesla T4


Mount Google Drive.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


1. Move all data into one consolidated all_data/ folder

In [ ]:
import os
import shutil

def move_all_data(train_dir, test_dir, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    classes = os.listdir(train_dir)

    for cls in classes:
        cls_output_dir = os.path.join(output_dir, cls)
        os.makedirs(cls_output_dir, exist_ok=True)

        for source_dir in [train_dir, test_dir]:
            cls_dir = os.path.join(source_dir, cls)
            if not os.path.exists(cls_dir):
                continue

            for fname in os.listdir(cls_dir):
                src = os.path.join(cls_dir, fname)
                dst = os.path.join(cls_output_dir, fname)
                shutil.move(src, dst)

train_dir = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/train'
test_dir = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/test'
all_data_dir = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/all_data'

move_all_data(train_dir, test_dir, all_data_dir)


2. Split the moved data into train/val/test

In [ ]:
import os
import shutil
import random
from pathlib import Path

def split_dataset(base_dir, output_dir, train_split=0.7, val_split=0.15, seed=42):
    random.seed(seed)
    classes = os.listdir(base_dir)

    for cls in classes:
        images = os.listdir(os.path.join(base_dir, cls))
        random.shuffle(images)

        total = len(images)
        train_end = int(total * train_split)
        val_end = int(total * (train_split + val_split))

        splits = {
            'train': images[:train_end],
            'val': images[train_end:val_end],
            'test': images[val_end:]
        }

        for split, split_images in splits.items():
            split_dir = os.path.join(output_dir, split, cls)
            os.makedirs(split_dir, exist_ok=True)
            for img in split_images:
                src = os.path.join(base_dir, cls, img)
                dst = os.path.join(split_dir, img)
                shutil.copy(src, dst)

original_dataset = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/all_data'
output_dataset = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/split'
split_dataset(original_dataset, output_dataset)


original_dataset = all_data_dir
output_dataset = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/split'

split_dataset(original_dataset, output_dataset)


 Fine-tune EfficientNetB0 on Famous Places Dataset

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

data_dir = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/split'
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')
test_dir = os.path.join(data_dir, 'test')

img_size = (224, 224)
batch_size = 32

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

checkpoint_dir = '/content/drive/MyDrive/ColabNotebooks/results/EfficientNet_FamousPlaces'
os.makedirs(checkpoint_dir, exist_ok=True)

checkpoint_path = os.path.join(checkpoint_dir, 'best_model.h5')
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
checkpoint = ModelCheckpoint(checkpoint_path, monitor='val_loss', save_best_only=True)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping, checkpoint],
    verbose=2
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f" Initial test accuracy: {test_accuracy * 100:.2f}%")

N = 5  
base_model.trainable = True
for layer in base_model.layers[:-N]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

fine_tune_checkpoint_path = os.path.join(checkpoint_dir, 'fine_tuned_model.h5')
checkpoint_fine = ModelCheckpoint(fine_tune_checkpoint_path, monitor='val_loss', save_best_only=True)

history_fine = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator,
    callbacks=[early_stopping, checkpoint_fine],
    verbose=2
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Final test accuracy (EfficientNetB0): {test_accuracy * 100:.2f}%")


Found 4250 images belonging to 50 classes.
Found 900 images belonging to 50 classes.
Found 950 images belonging to 50 classes.
16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


133/133 - 131s - 981ms/step - accuracy: 0.0165 - loss: 3.9471 - val_accuracy: 0.0200 - val_loss: 3.9165
Epoch 2/10


133/133 - 81s - 613ms/step - accuracy: 0.0176 - loss: 3.9340 - val_accuracy: 0.0200 - val_loss: 3.9153
Epoch 3/10


...

133/133 - 82s - 620ms/step - accuracy: 0.0221 - loss: 3.9476 - val_accuracy: 0.0200 - val_loss: 3.9105
Epoch 7/10


133/133 - 83s - 622ms/step - accuracy: 0.0188 - loss: 3.9463 - val_accuracy: 0.0200 - val_loss: 3.9097
Epoch 8/10


133/133 - 78s - 586ms/step - accuracy: 0.0172 - loss: 3.9366 - val_accuracy: 0.0211 - val_loss: 3.9095
Epoch 9/10


133/133 - 81s - 610ms/step - accuracy: 0.0169 - loss: 3.9454 - val_accuracy: 0.0200 - val_loss: 3.9091
Epoch 10/10


133/133 - 78s - 586ms/step - accuracy: 0.0191 - loss: 3.9374 - val_accuracy: 0.0167 - val_loss: 3.9088
30/30 ━━━━━━━━━━━━━━━━━━━━ 9s 315ms/step - accuracy: 0.0182 - loss: 3.9225
Final test accuracy (EfficientNetB0): 2.53%


In [ ]:
import os

def count_images_per_class(base_dir):
    classes = os.listdir(base_dir)

    class_counts = {}

    for cls in classes:
        class_dir = os.path.join(base_dir, cls)
        if os.path.isdir(class_dir): 
            image_files = os.listdir(class_dir)
            class_counts[cls] = len(image_files) 

    return class_counts

dataset_dir = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/all_data'
class_counts = count_images_per_class(dataset_dir)

for cls, count in class_counts.items():
    print(f"Class: {cls}, Number of images: {count}")


Class: Osaka_Castle, Number of images: 122
Class: The_Great_Sphinx, Number of images: 122
Class: Dresden_Frauenkirche, Number of images: 122
Class: Guggenheim_Museum, Number of images: 122
Class: Helsinki_Cathedral, Number of images: 122
Class: Chateau_Frontenac, Number of images: 122
Class: Millau_Bridge, Number of images: 122
Class: Brandenburg_Gate, Number of images: 122
Class: Hagia_Sophia, Number of images: 122
Class: Flatiron_Building, Number of images: 122
Class: Milan_Cathedral, Number of images: 122
Class: Mont_St_Michel, Number of images: 122
Class: Berlin_Museum_Island, Number of images: 122
Class: Neuschwanstein_Castle, Number of images: 122
Class: Leaning_Tower_of_Pisa, Number of images: 122
Class: Ephesus, Number of images: 122
Class: Sagrada_Familia, Number of images: 122
Class: Giants_Causeway, Number of images: 122
Class: Eiffel_Tower, Number of images: 122
Class: Blue_Mosque, Number of images: 122
Class: White_House, Number of images: 122
Class: Washington_Monument, N

Step 1: Resplit Data (50/50)

In [ ]:
import os
import shutil
import random
from pathlib import Path

def split_dataset(base_dir, output_dir, train_split=0.5, seed=42):
    random.seed(seed)
    classes = os.listdir(base_dir)

    os.makedirs(os.path.join(output_dir, 'train'), exist_ok=True)
    os.makedirs(os.path.join(output_dir, 'test'), exist_ok=True)

    for cls in classes:
        images = os.listdir(os.path.join(base_dir, cls))
        random.shuffle(images)

        total = len(images)
        train_end = int(total * train_split)

        splits = {
            'train': images[:train_end],
            'test': images[train_end:]
        }

        for split, split_images in splits.items():
            split_dir = os.path.join(output_dir, split, cls)
            os.makedirs(split_dir, exist_ok=True)
            for img in split_images:
                src = os.path.join(base_dir, cls, img)
                dst = os.path.join(split_dir, img)
                shutil.copy(src, dst)

original_dataset = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/all_data'
output_dataset = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/split'
split_dataset(original_dataset, output_dataset)


Step 2: Start EfficientNet Training (Fine-tuning a Few Layers)

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

train_dir = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/split/train'
test_dir = '/content/drive/MyDrive/ColabNotebooks/data/famous_places/split/test'
image_size = (224, 224)
batch_size = 32
num_classes = 50 

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=20, width_shift_range=0.2,
                                   height_shift_range=0.2, shear_range=0.2, zoom_range=0.2,
                                   horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_dir, target_size=image_size,
                                                    batch_size=batch_size, class_mode='categorical')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=image_size,
                                                  batch_size=batch_size, class_mode='categorical')

base_model = tf.keras.applications.EfficientNetB0(weights='imagenet', include_top=False,
                                                   input_shape=(224, 224, 3))

base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(1024, activation='relu'),
    layers.Dropout(0.2),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
              loss='categorical_crossentropy', metrics=['accuracy'])

lr_scheduler = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.1,  
    patience=3, 
    min_lr=1e-6,  
    verbose=1 
)

checkpoint_callback = ModelCheckpoint(
    '/content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model.h5', 
    monitor='val_loss', 
    save_best_only=True, 
    verbose=1,  
    save_weights_only=False  
)

history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[lr_scheduler, checkpoint_callback] 
)

base_model.trainable = True
fine_tune_at = 12

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss='categorical_crossentropy', metrics=['accuracy'])

history_fine_tune = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
    callbacks=[lr_scheduler, checkpoint_callback]  
)

model.save('/content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_model_fine_tuned.h5')

test_loss, test_accuracy = model.evaluate(test_generator)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Found 3050 images belonging to 50 classes.
Found 3050 images belonging to 50 classes.
Epoch 1/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 632ms/step - accuracy: 0.0236 - loss: 3.9434
Epoch 1: val_loss improved from inf to 3.91664, saving model to /content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model.h5


96/96 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - accuracy: 0.0235 - loss: 3.9434 - val_accuracy: 0.0200 - val_loss: 3.9166 - learning_rate: 1.0000e-04
Epoch 2/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 536ms/step - accuracy: 0.0187 - loss: 3.9304
Epoch 2: val_loss did not improve from 3.91664
96/96 ━━━━━━━━━━━━━━━━━━━━ 70s 728ms/step - accuracy: 0.0187 - loss: 3.9305 - val_accuracy: 0.0197 - val_loss: 3.9171 - learning_rate: 1.0000e-04
Epoch 3/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 541ms/step - accuracy: 0.0197 - loss: 3.9278
Epoch 3: val_loss improved from 3.91664 to 3.91536, saving model to /content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model.h5


96/96 ━━━━━━━━━━━━━━━━━━━━ 72s 753ms/step - accuracy: 0.0196 - loss: 3.9279 - val_accuracy: 0.0200 - val_loss: 3.9154 - learning_rate: 1.0000e-04
Epoch 4/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 539ms/step - accuracy: 0.0223 - loss: 3.9208
Epoch 4: val_loss improved from 3.91536 to 3.91468, saving model to /content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model.h5


96/96 ━━━━━━━━━━━━━━━━━━━━ 83s 764ms/step - accuracy: 0.0222 - loss: 3.9208 - val_accuracy: 0.0213 - val_loss: 3.9147 - learning_rate: 1.0000e-04
Epoch 5/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 543ms/step - accuracy: 0.0206 - loss: 3.9207
Epoch 5: val_loss improved from 3.91468 to 3.91339, saving model to /content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model.h5


96/96 ━━━━━━━━━━━━━━━━━━━━ 72s 752ms/step - accuracy: 0.0206 - loss: 3.9207 - val_accuracy: 0.0200 - val_loss: 3.9134 - learning_rate: 1.0000e-04
Epoch 6/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 533ms/step - accuracy: 0.0185 - loss: 3.9219
Epoch 6: val_loss improved from 3.91339 to 3.91325, saving model to /content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model.h5


...

96/96 ━━━━━━━━━━━━━━━━━━━━ 78s 807ms/step - accuracy: 0.2892 - loss: 3.3421 - val_accuracy: 0.4089 - val_loss: 3.1763 - learning_rate: 1.0000e-05
Epoch 8/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 564ms/step - accuracy: 0.3427 - loss: 3.1562
Epoch 8: val_loss improved from 3.17634 to 2.97072, saving model to /content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model.h5


96/96 ━━━━━━━━━━━━━━━━━━━━ 81s 804ms/step - accuracy: 0.3430 - loss: 3.1557 - val_accuracy: 0.4800 - val_loss: 2.9707 - learning_rate: 1.0000e-05
Epoch 9/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 556ms/step - accuracy: 0.4208 - loss: 2.9506
Epoch 9: val_loss improved from 2.97072 to 2.74848, saving model to /content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model.h5


96/96 ━━━━━━━━━━━━━━━━━━━━ 79s 768ms/step - accuracy: 0.4209 - loss: 2.9502 - val_accuracy: 0.5331 - val_loss: 2.7485 - learning_rate: 1.0000e-05
Epoch 10/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 557ms/step - accuracy: 0.4588 - loss: 2.7641
Epoch 10: val_loss improved from 2.74848 to 2.53331, saving model to /content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model.h5


96/96 ━━━━━━━━━━━━━━━━━━━━ 76s 791ms/step - accuracy: 0.4588 - loss: 2.7636 - val_accuracy: 0.5784 - val_loss: 2.5333 - learning_rate: 1.0000e-05


96/96 ━━━━━━━━━━━━━━━━━━━━ 19s 197ms/step - accuracy: 0.5695 - loss: 2.5629
Test Loss: 2.533306360244751
Test Accuracy: 0.5783606767654419


Unfreez more layers for better accuracy

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

model = tf.keras.models.load_model('/content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_model_fine_tuned.h5')

base_model = model.layers[0]

base_model.trainable = True
fine_tune_at = 20  

for layer in base_model.layers[:fine_tune_at]:
    layer.trainable = False

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss='categorical_crossentropy', metrics=['accuracy'])

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=50, 
    width_shift_range=0.4,
    height_shift_range=0.4, 
    shear_range=0.4,  
    zoom_range=0.4,  
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=[0.3, 1.7],  
    channel_shift_range=70.0,  
    vertical_flip=True  
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    '/content/drive/MyDrive/ColabNotebooks/data/famous_places/split/train', 
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    '/content/drive/MyDrive/ColabNotebooks/data/famous_places/split/test',  
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

history_fine_tune = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator,
        callbacks=[
        ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.1,
            patience=3,
            min_lr=1e-6,
            verbose=1
        ),
        ModelCheckpoint(
            '/content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model_fine_tuned.keras',
            monitor='val_loss',
            save_best_only=True,
            verbose=1,
            save_weights_only=False
        )
    ]
)

model.save('/content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_model_fine_tuned_v2.keras') 

test_loss, test_accuracy = model.evaluate(test_generator)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")


Found 3050 images belonging to 50 classes.
Found 3050 images belonging to 50 classes.
Epoch 1/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 933ms/step - accuracy: 0.1001 - loss: 3.6512
Epoch 1: val_loss improved from inf to 4.35794, saving model to /content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model_fine_tuned.keras
96/96 ━━━━━━━━━━━━━━━━━━━━ 184s 1s/step - accuracy: 0.1001 - loss: 3.6509 - val_accuracy: 0.0289 - val_loss: 4.3579 - learning_rate: 1.0000e-05
Epoch 2/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 660ms/step - accuracy: 0.1111 - loss: 3.5774
Epoch 2: val_loss improved from 4.35794 to 3.91760, saving model to /content/drive/MyDrive/ColabNotebooks/data/famous_places/efficientnet_best_model_fine_tuned.keras
96/96 ━━━━━━━━━━━━━━━━━━━━ 86s 890ms/step - accuracy: 0.1111 - loss: 3.5771 - val_accuracy: 0.0302 - val_loss: 3.9176 - learning_rate: 1.0000e-05
Epoch 3/10
96/96 ━━━━━━━━━━━━━━━━━━━━ 0s 653ms/step - accuracy: 0.1379 - loss: 3.4634
Epoch 3: val_loss improved from 3.9176